# Data cleaning and aggregation

In this script we will clean the given datasets, and import them to the local SQLite database.

| atribute           | info                                                                                                                         | source | attributes| year |
| ------------------ | ---------------------------------------------------------------------------------------------------------------------------- | ------ | ---- | ---- |
| year | x | teams | attributes  | current |
| tmID | x | teams | attributes  | current |
| playoff | x | teams | attributes  | current |
| averageWinRate | If previous year is not available, use the yearly average for the selected attributes. | teams_players | won, lost  | previous |
| averagePoints | If previous year is not available, use the yearly average for the selected attributes. | teams_players | points  | previous |
| averageRebounds | If previous year is not available, use the yearly average for the selected attributes. | teams_players | rebounds  | previous |
| averageAssists | If previous year is not available, use the yearly average for the selected attributes. | teams_players | assists  | previous |
| averageSteals | If previous year is not available, use the yearly average for the selected attributes. | teams_players | steals  | previous |
| averageBlocks | If previous year is not available, use the yearly average for the selected attributes. | teams_players | blocks  | previous |
| averageTurnovers | If previous year is not available, use the yearly average for the selected attributes. | teams_players | turnovers  | previous |
| averageFGRatio | If previous year is not available, use the yearly average for the selected attributes. | teams_players | fgAttempted, fgMade  | previous |
| averageFTRatio | If previous year is not available, use the yearly average for the selected attributes. | teams_players | ftAttempted, ftMade  | previous |
| averageThreeRatio | If previous year is not available, use the yearly average for the selected attributes. | teams_players | threeAttempted, threeMade  | previous |
| coachWinRate | If previous year is not available, use the yearly average for the selected attributes. | coaches | won, lost  | previous |
| numberOfAwardedPlayers | If previous year is not available, use the yearly average for the selected attributes. | awards_players | *count  | previous |

SQL Not used

In [1]:
# Create connection layer

import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.row_factory = sqlite3.Row
    except Error as e:
        print(e)

    return conn
        
def delete_all_data(conn):
    cur = conn.cursor()
    print("Deleted ", cur.execute("DELETE FROM coaches").rowcount, " from coaches table.")
    print("Deleted ", cur.execute("DELETE FROM awards_players").rowcount, " from awards_players table.")
    print("Deleted ", cur.execute("DELETE FROM players_teams").rowcount, " from players_teams table.")
    print("Deleted ", cur.execute("DELETE FROM series_post").rowcount, " from series_post table.")
    print("Deleted ", cur.execute("DELETE FROM teams_post").rowcount, " from teams_post table.")
    print("Deleted ", cur.execute("DELETE FROM teams").rowcount, " from teams table.")
    print("Deleted ", cur.execute("DELETE FROM players").rowcount, " from players table.")


database = "./datasets/original/sqlite_database_g79.db"

# create a database connection
conn = create_connection(database)

In [2]:
# Query test
def get_teams(conn):
  cur = conn.cursor()
  cur.execute("SELECT * FROM teams LIMIT 10")
  
  return cur.fetchall()

# --------------------------------------------------------------

allTeams = get_teams(conn)

for i in allTeams:
  print(i["name"])

Atlanta Dream
Atlanta Dream
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Chicago Sky


In [10]:
delete_all_data(conn)

Deleted  162  from coaches table.
Deleted  95  from awards_players table.
Deleted  1876  from players_teams table.
Deleted  70  from series_post table.
Deleted  80  from teams_post table.
Deleted  142  from teams table.
Deleted  893  from players table.


In [3]:
# Close connection 

conn.close()

## Cleaning and aggregation

Initialize all csv paths and selected attributes.

In [2]:
players_teams_csv_file_path = "./datasets/original/players_teams.csv"
coaches_csv_file_path = "./datasets/original/coaches.csv"
teams_csv_file_path = "./datasets/original/teams.csv"
awards_players_csv_file_path = "./datasets/original/awards_players.csv"

aggregated_table_csv_file_path = "./datasets/generated/aggregated_table.csv"

players_teams_selected_attributes = ["playerID", "year", "tmID", "points", "rebounds", "assists", "steals", "blocks", "turnovers", "fgAttempted", "fgMade", "ftAttempted", "ftMade", "threeAttempted", "threeMade"]
coaches_selected_attributes = ["coachID", "year", "tmID", "stint", "won", "lost"]
teams_selected_attributes = ["year", "tmID", "won", "lost", "playoff"]
awards_players_selected_attributes = ["playerID", "year"]

aggregated_attributes = ["tmID", "year", "playoff", "averageWinRate", "averagePoints", "averageRebounds", "averageAssists", "averageSteals", "averageBlocks", "averageTurnovers", "averageFGRatio", "averageFTRatio", "averageThreeRatio", "coachWinRate", "numberOfAwardedPlayers", "listOfPlayers"]


Load all data and select only wanted attributes from DataFrames.

In [3]:
import pandas as pd

initial_df_teams_players = pd.read_csv(players_teams_csv_file_path)
df_teams_players = initial_df_teams_players[players_teams_selected_attributes]

initial_df_coaches = pd.read_csv(coaches_csv_file_path)
df_coaches = initial_df_coaches[coaches_selected_attributes]

initial_df_teams = pd.read_csv(teams_csv_file_path)
df_teams = initial_df_teams[teams_selected_attributes]

initial_df_awards_players = pd.read_csv(awards_players_csv_file_path)
df_awards_players = initial_df_awards_players[awards_players_selected_attributes]


In [4]:
import pandas as pd

df_final_temp = df_teams[["tmID", "year", "playoff"]]
df_final_temp = df_final_temp.sort_values(["tmID", "year"], ascending=False)

df_final_temp["players"] = "" 
df_final_temp = df_final_temp.reset_index()
df_final_temp = df_final_temp.drop("index", axis=1)
df_final_temp.head()

for index, row in df_final_temp.iterrows():
  currentYear = row["year"]
  currentTeamId = row["tmID"]
  
  mask = df_teams_players[(df_teams_players["year"] == currentYear ) & (df_teams_players["tmID"] == currentTeamId)]
  
  df_final_temp["players"][index] = mask["playerID"].values


df_final_temp.head(20)

C:\Users\Zan\AppData\Local\Temp\ipykernel_5700\3599968181.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_temp["players"][index] = mask["playerID"].values


,tmID,year,playoff,players
0,WAS,10,Y,"[ajavoma01w, beardal01w, blueni01w, colemma01w..."
1,WAS,9,N,"[beardal01w, blueni01w, currimo01w, gardnan01w..."
2,WAS,8,N,"[beardal01w, blueni01w, currimo01w, goringi01w..."
3,WAS,7,Y,"[beardal01w, blueni01w, byearla01w, jamesta01w..."
4,WAS,6,N,"[amachma01w, beardal01w, brownki01w, choneka01..."
5,WAS,5,Y,"[beardal01w, brownki01w, choneka01w, dalesst01..."
6,WAS,4,N,"[brownki01w, burgean01w, dalesst01w, henniso01..."
7,WAS,3,Y,"[brownki01w, bullevi01w, burgean01w, dalesst01..."
8,WAS,2,N,"[aldrima01w, bauerca01w, bullevi01w, burgean01..."
9,WAS,1,Y,"[aldrima01w, anderke01w, bullevi01w, campbmi01..."


Calculate all the averages for all tables, so we can populate missing values later.

In [5]:
df_teams_players_average = df_teams_players.groupby("year").agg("mean", numeric_only=True)
df_teams_average = df_teams.groupby("year").agg("mean", numeric_only=True)
df_coaches_average = df_coaches.groupby("year").agg("mean", numeric_only=True)

df_teams_players_average = df_teams_players_average.reset_index()
df_teams_average = df_teams_average.reset_index()
df_coaches_average = df_coaches_average.reset_index()

df_teams_average.head(10)


,year,won,lost
0,1,16.0,16.0
1,2,16.0,16.0
2,3,16.0,16.0
3,4,17.0,17.0
4,5,17.0,17.0
5,6,17.0,17.0
6,7,17.0,17.0
7,8,17.0,17.0
8,9,17.0,17.0
9,10,17.0,17.0


In [6]:
year_offset = 1

# loop all temp rows
df_final = df_final_temp
new_columns = ["averageWinRate", "averagePoints", "averageRebounds", "averageAssists", "averageSteals", "averageBlocks", "averageTurnovers", "averageFGRatio", "averageFTRatio", "averageThreeRatio", "coachWinRate", "numberOfAwardedPlayers"]
df_final[new_columns] = ""

for idx in range(len(df_final)):
  row = df_final.iloc[idx]
    
  currentYear = row["year"]
  currentTeamId = row["tmID"]
  players = row["players"]
  coach_id = df_coaches[(df_coaches["tmID"] == currentTeamId) & (df_coaches["year"] == currentYear)]["coachID"].values[0]
  
  if(currentYear == 1):
    continue
  
  previousYearExists = any((df_final["year"] == currentYear - year_offset) & (df_final["tmID"] == currentTeamId))
  
  # for all players get data from currentYear - year_offset
  players_data = df_teams_players[(df_teams_players["playerID"].isin(players)) & (df_teams_players["year"] == currentYear - year_offset)]
  teams_data = df_teams[(df_teams["tmID"] == currentTeamId) & (df_teams["year"] == currentYear - year_offset)]
  coach_data = df_coaches[(df_coaches["coachID"] == coach_id) & (df_coaches["year"] == currentYear - year_offset)]
  awards_data = df_awards_players[(df_awards_players["playerID"].isin(players)) & (df_awards_players["year"] < currentYear)]
  
  # Populate missing teams_players data
  missing_players = []
  
  for player in list(players):
    if not player in players_data["playerID"].values:
      missing_players.append(player)
  
  last_year_players_average = df_teams_players_average[df_teams_players_average["year"] == (currentYear - year_offset)]
  last_year_players_average["tmID"] = currentTeamId
  
  # populate players_data with avg players data
  for player in missing_players:
    last_year_players_average["playerID"] = player
    players_data = players_data.append(last_year_players_average)
    
  # Populate missing coaches data
  if len(coach_data) == 0:
    last_year_coaches_average = df_coaches_average[df_coaches_average["year"] == (currentYear - year_offset)]
    last_year_coaches_average["tmID"] = currentTeamId
    last_year_coaches_average["coachID"] = coach_id
    
    coach_data = coach_data.append(last_year_coaches_average)
  
  # Populate missing teams data
  if len(teams_data) == 0:
    last_year_teams_average = df_teams_average[df_teams_average["year"] == (currentYear - year_offset)]
    last_year_teams_average["tmID"] = currentTeamId
    
    teams_data = teams_data.append(last_year_teams_average)
  
  # calculate attributes
  averagePoints = players_data["points"].mean()
  averageRebounds = players_data["rebounds"].mean()
  averageAssists = players_data["assists"].mean()
  averageSteals = players_data["steals"].mean()
  averageBlocks = players_data["blocks"].mean()
  averageTurnovers = players_data["turnovers"].mean()
  averageFGRatio = (players_data["fgMade"] / players_data["fgAttempted"]).mean()
  averageFTRatio = (players_data["ftMade"] / players_data["ftAttempted"]).mean()
  averageThreeRatio = (players_data["threeMade"] / players_data["threeAttempted"]).mean()

  averageWinRate = (teams_data["won"] / (teams_data["won"] + teams_data["lost"])).mean()
  coachWinRate = (coach_data["won"] / (coach_data["won"] + coach_data["lost"])).mean()
  numberOfAwardedPlayers = len(awards_data)
  
  df_final["averagePoints"][idx] = averagePoints
  df_final["averageRebounds"][idx] = averageRebounds
  df_final["averageAssists"][idx] = averageAssists
  df_final["averageSteals"][idx] = averageSteals
  df_final["averageBlocks"][idx] = averageBlocks
  df_final["averageTurnovers"][idx] = averageTurnovers
  df_final["averageFGRatio"][idx] = averageFGRatio
  df_final["averageFTRatio"][idx] = averageFTRatio
  df_final["averageThreeRatio"][idx] = averageThreeRatio
  df_final["averageWinRate"][idx] = averageWinRate
  df_final["coachWinRate"][idx] = coachWinRate
  df_final["numberOfAwardedPlayers"][idx] = numberOfAwardedPlayers
  
df_final = df_final[df_final["year"] > year_offset]
df_final = df_final.drop("players", axis=1)

# write to JSON
df_json = df_final.to_json(orient="records")

new_file = open("./datasets/generated/cleaned_aggregated_data.json", "w")
new_file.writelines(df_json)
new_file.close()

df_final.head(10)


C:\Users\Zan\AppData\Local\Temp\ipykernel_5700\3844699177.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_year_players_average["tmID"] = currentTeamId
C:\Users\Zan\AppData\Local\Temp\ipykernel_5700\3844699177.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_year_players_average["playerID"] = player
C:\Users\Zan\AppData\Local\Temp\ipykernel_5700\3844699177.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

,tmID,year,playoff,averageWinRate,averagePoints,averageRebounds,averageAssists,averageSteals,averageBlocks,averageTurnovers,averageFGRatio,averageFTRatio,averageThreeRatio,coachWinRate,numberOfAwardedPlayers
0,WAS,10,Y,0.470588,222.360859,101.006193,40.109001,22.393889,10.068126,42.180842,0.409265,0.7665,0.253256,0.5,0
1,WAS,9,N,0.529412,229.81875,101.2125,52.352083,24.16875,9.05625,46.260417,0.430233,0.77518,0.32657,0.5,2
2,WAS,8,N,0.470588,208.948315,87.485393,40.890637,23.185019,8.444944,38.629213,0.421434,0.746699,0.342584,0.470588,2
3,WAS,7,Y,0.5,234.080838,98.093812,57.265469,25.990519,12.104291,47.100798,0.423451,0.748017,0.347804,0.4375,2
4,WAS,6,N,0.264706,208.569231,96.557265,45.564103,22.707692,11.046154,40.806838,0.398834,0.69454,0.337006,0.470588,1
5,WAS,5,Y,0.53125,222.403226,99.341782,47.374424,21.371736,7.15361,43.255376,0.417046,0.758448,0.354621,0.5,1
6,WAS,4,N,0.3125,169.14217,85.554945,45.172734,23.415179,9.210165,41.109547,0.38955,0.700562,0.312347,0.5,1
7,WAS,3,Y,0.4375,183.479487,88.784615,34.125641,20.7,11.566667,37.34359,0.424849,0.735209,0.332947,0.5,0
10,UTA,3,Y,0.5625,219.261095,100.005917,43.781805,19.875,14.971154,45.162722,0.438473,0.748172,0.305888,0.5,0
13,SEA,10,Y,0.5,224.099155,108.139618,46.85727,21.895064,14.612717,45.646065,0.416928,0.737295,0.23731,0.5,5
